# JSON format for multidimensional data
version : 2024-02-28

## Introduction
This memo is a proposal to implement a compact and reversible (lossless round-trip) JSON interface for multidimensional data and in particular for Numpy (see issue #12481).

The [JSON-NTV](https://www.ietf.org/archive/id/draft-thomy-json-ntv-02.html) (Named and Typed value) format is a JSON format which integrates a notion of type .
    
In particular, it makes it possible to provide a reversible (lossless round-trip) interface for multidimensional data.
     
This format has also been implemented for [tabular data](https://www.ietf.org/archive/id/draft-thomy-ntv-tab-00.html) (see NTV-pandas package available in the [pandas ecosystem](https://pandas.pydata.org/community/ecosystem.html) and the [PDEP12](https://pandas.pydata.org/pdeps/0012-compact-and-reversible-JSON-interface.html) specification). .

This memo presents a possible implementation for multidimensional data and in particular for Numpy.

## Benefits
The use of this format has the following advantages:

- Taking into account data types not known to Numpy,
- Reversible format
- Interoperability with other tools for tabular or multidimensional data (e.g. pandas, Xarray)
- Ease of sharing Json format
- Binary coding possible (e.g. CBOR format)
- Format integrating data of different nature


## NTV data
NTV format is a data representation with three attributes:
- NTVname (string)
- NTVtype (enumerate string)
- NTVvalue (JSON object) 

Two entities are defined:
- NTVlist : ordered list of entities
- NTVsingle : entity not composed with other entities

The JSON representation of NTVsingle entities is :
- value :
```json
    25, 'test', [1,2]
```

- name and value : 
```json
    {'test': 25}, {'test:': [1,2]}
```
- type and value :
```json
    {':day': 25}, {':point': [1,2]}
```
- type name and value : 
```json
    {'equinox:date': '2023-09-23'}, {'paris:point': [2.35, 48.86]}
```

The JSON representation of NTVlist entities is :

- { 'name_NTVlist:type_NTVlist': { JSON_entity1, ... JSONentityn } } if entities have JSON_member representation 
```json
    { 'example': {'equinox:date': '2023-09-23', 'paris:point': [2.35, 48.86] }}
```

- { 'name_NTVlist:type_NTVlist': [ JSON_entity1, ... JSONentityn ] } in the other cases
```json
    { 'example': [25, {'paris:point': [2.35, 48.86] }, 'test']}
```    

## Multidimensional data

### Multidimensional types
To take multidimensional data into account, two types are added (see [Appendix](#Appendix---JSON-representation)):

- `ndarray` (multidimensional array) : simple N-dimensional arrays of homogeneous data types 
- `xndarray` (multidimensional array with axes): N-dimensional arrays with additional data

Numpy.array data (data + dtype) corresponds to ndarray format.
The Xarray.DataArray data corresponds to the xndarray format.


In [1]:
import numpy as np
from pprint import pprint
from json_ntv import Ntv
import pandas as pd
from shapely.geometry import Point
import ntv_pandas as npd
from numpy_ntv_connector import read_json, read_json_tab, to_json, to_json_tab

#### Simple Ndarray (type 'ndarray')

In [2]:
ex = np.arange(1, 7).reshape((2, 3))

print("example (with and without dtype) :\n")
print(to_json(ex))
print(to_json(ex, extension='kg'))
print(to_json(ex, notype=True))

print("\nreversibility :\n")
ex2 = read_json(to_json(ex), header=False)
print(np.array_equal(ex2, ex))

example (with and without dtype) :

{':ndarray': ['int32', [2, 3], [1, 2, 3, 4, 5, 6]]}
{':ndarray': ['int32[kg]', [2, 3], [1, 2, 3, 4, 5, 6]]}
{':ndarray': [[2, 3], [1, 2, 3, 4, 5, 6]]}

reversibility :

True


#### Axed Ndarray (type 'xndarray')
This type is associated to a Ndarray with additional data.

- dims : list of name of each axis
- coords : dict of 1-dimensional ndarray for each axis

Example
```json
{'example1:xndarray':
    {'data'  : ['int32', [2, 3, 2], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]], 
     'dims'   : ['x', 'y', 'option'],
     'coords' : {
         'x': [['x1', 'x2']] , 
         'y': [['y1', 'y2', 'y3']],
         'option': [[True, False]]}
}
```

In [3]:
# example with axes names and axes variables
a = np.arange(1, 13).reshape((2, 3, 2))
add = {'dims'   : ['x', 'y', 'option'],
       'coords' : {
         'x': [['x1', 'x2']] , 
         'y': [['y1', 'y2', 'y3']],
         'option': [[True, False]]}}

print("\nexample with additional data :\n")
pprint(to_json(a, add=add, name='test', extension='kg'), width=150)

print("\nreversibility :\n")
a2, add2 = read_json(to_json(a, add=add), header=False)
print(np.array_equal(a2, a), add2 == add)


example with additional data :

{'test:xndarray': {'coords': {'option': [[True, False]], 'x': [['x1', 'x2']], 'y': [['y1', 'y2', 'y3']]},
                   'data': ['int32', [2, 3, 2], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]],
                   'dims': ['x', 'y', 'option']}}

reversibility :

True True


#### Full axed Ndarray (type 'xndarray')
Additional data can be non-axes variables (`xndarray` included in 'coords') or metadata (keyword `attrs`).

Example :

```json
{'example2:xndarray':
    {'data'  : ['int32', [2, 3, 2], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]], 
     'dims'   : ['x', 'y', 'option'],
     'coords' : {
         'x': [['x1', 'x2']] , 
         'y': ['string', ['y1', 'y2', 'y3']],
         'option': [[True, False]],
         'xy': {'dims': ['x','y'], 'data': [[2,3], ['x1y1','x1y2','x1y3','x2y1','x2y2','x2y3']]}, 
         'opt_num': {'dims': ['option'], 'data': [[0, 1]]}},
     'attrs'  : {'meta': 'everything'}}
}
```

### Format JSON
The JSON representation is obtained by:

- Converting structure of Numpy data into JSON structure
- Conversion of elementary data into a JSON primitive,
- Mapping the Numpy dtype and the nature of the object into an NTV type

These functions are similar to those used for the JSON-NTV representation of Pandas data ([see pandas examples](https://nbviewer.org/github/loco-philippe/ntv-pandas/blob/main/example/example_ntv_pandas.ipynb)).

See mapping example with `ndarray` below (with `xndarray` the NTVtype is included in the full JSON object).

#### Data with numpy dtype
The NTVtype is deduced from the dtype (int_xx, uint_xx, float_xx, bool, bytes, str, datetime64, timedelta).

```python
np.array([1, 2], dtype='int64')                               <-> ['int64', [1, 2]]
np.array([True, False], dtype='bool')                         <-> ['boolean', [True, False]]
np.array(['1+2j', 1], dtype='complex')                        <-> ['complex', ['1.+2.j', '1.0.j']]
np.array(['test1', 'test2'], dtype='str_')                    <-> ['string', ['test1', 'test2']]
np.array(['2022-01-01T10:05:21'], dtype='datetime64')         <-> ['datetime', ['2022-01-01T10:05:21']]
np.array(['2022-01-01', '2023-01-01'], dtype='datetime64[D]') <-> ['date', ['2022-01-01', '2023-01-01']]
np.array(['2022-01', '2023-01'], dtype='datetime64[M]')       <-> ['yearmonth', ['2022-01', '2023-01']]
np.array(['2022', '2023'], dtype='datetime64[Y]')             <-> ['year', ['2022', '2023']]
np.array([1,2], dtype = 'timedelta[D]')                       <-> ['timedelta[D]', [1, 2]]
np.array([b'abc\x09', b'abc'], dtype = 'bytes')               <-> ['binary', ['abc\x09', 'abc']]
```

#### Data with standard python type
The NTVtype is deduced from the python type (time, list, dict, None, decimal64).

```python
np.array([datetime.time(10, 2, 3)], dtype='object')   <-> ['time', ['10:02:03']]
np.array(pd.array([[1,2], [3,4]]), dtype='object')    <-> ['array', [[1,2], [3,4]]]
np.array([{'one':1}, {'two':2}], dtype='object')      <-> ['object', [{'one':1}, {'two':2}]]
np.array([None], dtype='object')                      <-> ['null', [None]]
np.array([Decimal('10.2')], dtype='object')           <-> ['decimal64', [10.2]]
```

#### Data with python object 
The NTVtype (ndarray, ntv, point, line, polygon, field, tab) is deduced from the type of the python objects.

```python
np.array([np.array([1, 2], dtype='int64'), np.array(['test1'], dtype='str_')], 
         dtype='object')                           <-> ['ndarray', [['int64', [1, 2]], 
                                                                    ['string', ['test1']]]]
ntv = np.empty(2, dtype='object')
ntv[:] = [Ntv.obj({':point':[1,2]}), NtvSingle(12, 'noon', 'hour')]
ntv                                                <-> ['ntv', [{":point": [1, 2]}, {"noon:hour": 12}]
np.array([Point([1,2]), Point([3,4])])             <-> ['point', [[1, 2], [3, 4]]]
np.array([LineaRing([[0, 0], [0, 1], [1, 1]])])    <-> ['line', [[0.,0.], [0.,1.], [1.,1.], [0.,0.]]]
np.array([pd.Series([1,2,3])])                     <-> ['field', {'test': [1, 2, 3]}]
np.array([pd.DataFrame({'::date': ['1964-01-01', '1985-02-05'], 
                        'names::string': ['john', 'eric']})])
                                                   <-> ['tab', {'::date': ['1964-01-01', '1985-02-05'],
                                                                'names::string': ['john', 'eric']}]
```

#### Other data
The NTVtype can't be inferred from the python type or the numpy dtype. It should be included in additional data.
It concerns: 

- type with extension (e.g. number with unit),
- number with generic format (json, int, float, number)
- number with semantic value (month, day, wday, yday, week, hour, minute, second)
- string with semantic value (base16, base32, base64, period, duration, jpointer, uri, uriref, iri, iriref, email, regex, hostname, ipv4, ipv6, file, geojson)
- object with semantic value (geometry, timearray)

Examples with tuple (name, data):

```python
( 'int64[kg]', np.array([[1, 2], [3,4]]) )                 <-> ['int64[kg]', [2,2], [1, 2, 3, 4]]
( 'int', np.array([[1, 2], [3,4]]) )                       <-> ['int', [2,2], [1, 2, 3, 4]]
( 'json', np.array([1, 'two']) )                           <-> ['json', [1, 'two']]
( 'month', np.array([1, 2]) )                              <-> ['month', [1, 2]]
( 'base16', np.array(['1F23', '236A5E']) )                 <-> ['base16', ['1F23', '236A5E']]
( 'duration', np.array(['P3Y6M4DT12H30M5S']) )             <-> ['duration', ['P3Y6M4DT12H30M5S']]
( 'uri', np.array(['geo:13.4125,103.86673']) )             <-> ['uri', ['geo:13.4125,103.86673']]
( 'email', np.array(['John Doe <jdoe@mac.example>']) )     <-> ['email', ['John Doe <jdoe@mac.example>']]
( 'ipv4', np.array(['192.168.1.1']) )                      <-> ['ipv4', ['192.168.1.1']]
```

### Using in NTV data
The NTV format makes it possible to group data of different types in the same structure or in specific structure ([see NTV overview](https://nbviewer.org/github/loco-philippe/NTV/blob/main/example/example_ntv.ipynb)).

#### Include Ndarray in NTV structure

In [4]:
nd1 = np.array([1, 2, 3, 4, 5, 6])
nd2 = np.array([1, 2, 3, 4])
nd3 = np.array([1, 2, 3, 4])
# simple
print('example NTVsingle (Json and object representation)\n')
simple = Ntv.obj({'simple:ndarray': nd1})
print(simple)
pprint(simple.to_obj(format='obj'))
# list
print('\nexample NTVlist (Json and object representation)\n')
a_list = Ntv.obj({'list::ndarray': [nd1, {'second ndarray': nd2}, nd3]})
print('\n' + str(a_list))
pprint(a_list.to_obj(format='obj'))
# mixte
print('\nexample NTVlist with mixed data (Json and object representation)\n')
mixte = Ntv.obj({'mixed': [nd2, {'coordinate':Point(1,2)}, {'pandas series': pd.Series([1,2,3])}]})
print('\n' + str(mixte))
pprint(mixte.to_obj(format='obj'), width=150)

example NTVsingle (Json and object representation)

{"simple:ndarray": ["int32", [1, 2, 3, 4, 5, 6]]}
{'simple': array([1, 2, 3, 4, 5, 6])}

example NTVlist (Json and object representation)


{"list::ndarray": [["int32", [1, 2, 3, 4, 5, 6]], {"second ndarray": ["int32", [1, 2, 3, 4]]}, ["int32", [1, 2, 3, 4]]]}
{'list::ndarray': [array([1, 2, 3, 4, 5, 6]),
                   {'second ndarray': array([1, 2, 3, 4])},
                   array([1, 2, 3, 4])]}

example NTVlist with mixed data (Json and object representation)


{"mixed": {":ndarray": ["int32", [1, 2, 3, 4]], "coordinate:point": [1.0, 2.0], "pandas series:field": [1, 2, 3]}}
{'mixed': [array([1, 2, 3, 4]), {'coordinate': <POINT (1 2)>}, {'pandas series': 0    1
1    2
2    3
dtype: int64}]}


#### Include Ndarray in other objects

In [5]:
sr = pd.Series([1, 2, np.array([1, 2, 3, 4])])
mixin = Ntv.obj({'mixin': sr})
print(mixin)


{"mixin:field": [1, 2, {":ndarray": ["int32", [1, 2, 3, 4]]}]}


### Equivalence of tabular format and multi-dimensional format
The conversion between the two tabular and multi-dimensional formats is simple and lossless.

We can therefore share data between a tabular tool and a multidimensional tool via this format.

#### Format conversion

In [6]:
print("example without axes :\n")
pprint(to_json(ex))
pprint(to_json_tab(ex))

print("\nexample with axes :\n")
pprint(to_json(a, add=add), width=150)
pprint(to_json_tab(a, add), width=150)

example without axes :

{':ndarray': ['int32', [2, 3], [1, 2, 3, 4, 5, 6]]}
{':tab': {'data::int32': [1, 2, 3, 4, 5, 6],
          'dim_0': [[0, 1], [3]],
          'dim_1': [[0, 1, 2], [1]]}}

example with axes :

{':xndarray': {'coords': {'option': [[True, False]], 'x': [['x1', 'x2']], 'y': [['y1', 'y2', 'y3']]},
               'data': ['int32', [2, 3, 2], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]],
               'dims': ['x', 'y', 'option']}}
{':tab': {'data::int32': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
          'option': [[True, False], [1]],
          'x': [['x1', 'x2'], [6]],
          'y': [['y1', 'y2', 'y3'], [2]]}}


#### Compatibility with tabular tools

In [7]:
print('pandas DataFrame :')
ex_df = npd.read_json(to_json_tab(ex))
display(ex_df)

a_df = npd.read_json(to_json_tab(a, add))
display(a_df)
print('dtypes:\n' + str(a_df.dtypes))

pandas DataFrame :


,dim_0,dim_1,data
0,0,0,1
1,0,1,2
2,0,2,3
3,1,0,4
4,1,1,5
5,1,2,6


,x,y,option,data
0,x1,y1,True,1
1,x1,y1,False,2
2,x1,y2,True,3
3,x1,y2,False,4
4,x1,y3,True,5
5,x1,y3,False,6
6,x2,y1,True,7
7,x2,y1,False,8
8,x2,y2,True,9
9,x2,y2,False,10


dtypes:
x         category
y         category
option    category
data         int32
dtype: object


### tabular data to multidimensional data
Tabular data can be analysed to identify the dimension and the fields associated to each axe

In [8]:
analys = a_df.npd.analysis(True)
print('dimension:\n', analys.dimension)
partition = analys.field_partition(mode='id')
print('\npartition:\n', partition)

dimension:
 3

partition:
 {'primary': ['x', 'y', 'option'], 'secondary': [], 'unique': [], 'variable': ['data']}


'primary' fields are converted into axes.

The shape is deduced from the length of axes (categorical format)

In [9]:
a_df_sort = a_df.sort_values(partition['primary'])
a3, add3 = read_json_tab(a_df_sort.npd.to_json(header=False, index=False))

print("\nreversibility :\n")
print(np.array_equal(a3, a), Ntv.obj(add3) == Ntv.obj(add))


reversibility :

True True


## Astropy specific points
This chapter presents some points related to the Astropy data structure that can be integrated into the JSON-NTV format.

### Units and quantities
- 'unit' is a specif type
- three options are available for quantities:

    - option 1 : add specific types including unit
    - option 2 : add unit as type extension for existing types
    - option 3 : including unit in the name

- Option 2 is retained 

    - This option can be extended to other usages. For example:
    
    ```
    {"comment:string[fr]": "Paris est une belle ville"}
    ```

    - This option is compatible with NTV structure. For example 
    
    ```
    {"list_of_ndarray::ndarray[kg]": { "array1": [1, 2, 3, 4], "array2": [5, 6, 7, 8]}}
    ```

In [10]:
ntv = Ntv.obj({"list_of_ndarray::ndarray[kg]": { "array1": [[2, 2], [1, 2, 3, 4]], "array2": [[2, 2], [5, 6, 7, 8]]}})
print('json representation :\n', ntv[0])
print('\nNdarray representation :\n', ntv[0].to_obj(format='obj', type=True))

json representation :
 {"array1:ndarray[kg]": [[2, 2], [1, 2, 3, 4]]}

Ndarray representation :
 {'array1:ndarray[kg]': array([[1, 2],
       [3, 4]])}


Example Unit

```
{'mass:unit': 'kg'}
```
    
Example Quantities

```
{'ex_simple:[m/s]':            0.47}
{'ex_simple_typ:float64[m/s]': 0.47}
{'ex_array:ndarray[m/s]':      [2., 2.5, 3., 3.5, 4., 4.5, 5.]}
{'ex_array_typ:ndarray[m/s]':  ['float64', [2., 2.5, 3., 3.5, 4., 4.5, 5.]]}
```

### Coordinates
The existing 'point' type (and also the other types: pointstr, line, polygon, multipolygon, box...) can be used with the coordinate object (perhaps with a type extension). e.g.

```
{':point[icrs]' : [ 10.625, 41.2] }
```

### Table
The `Table` or `Qtable` object is represented using the `tab` format dedicated to tabular structures. 

```
{'ex_table:tab': {'x:string': ['x1', 'x1', 'x2', 'x2'], 
                  'y:string': ['y1', 'y2', 'y1', 'y2'], 
                  'value:float64[kg]': [1.0, 2.0, 3.0, 4.0],
                  'meta': 'everything'}
```

This ensures interoperability between tabular tools (e.g. Pandas).


### Other structures

The other structures were not examined. They can be integrated using the following tools:

- Addition of new types: for types having a transversal character
- Adding type via a Namespace: for types associated with specific data (e.g. “astro.xxx)
- Definition of an imposed structure for a given type
- Using specific type extension


## Appendix - JSON representation

### Multidimension
Multidimensional data is expressed with the JSON structure as a nested JSON-array. 
This representation has an drawback: It is not possible to include objects represented by an array in this structure.

    e.g. What is the JSON representation of this 1-dimension ndarray : array([list([1, 2]), list([3, 4])], dtype=object) ? 

A first solution is to use a JSON-object instead of a JSON-array :

```json
    [{":array":[1, 2]}, {":array":[3, 4]}]
```

A second solution is to convert an array to a string :

```json
    ["[1, 2]", "[3, 4]"]
```

A third solution is to represent the shape and the flattened array :

```json
    [[2], [1, 2, 3, 4]]
```       


### Type representation

Another constraint is to represent the data type. Three options :

option 1 : use NTV representation  -> The examples above will become

```json
    {":int32": [{":array":[1, 2]}, {":array":[3, 4]}]}
    {":int32": ["[1, 2]", "[3, 4]"]}
    {":int32": [[2], [1, 2, 3, 4]]}
```

option 2 : use JSON-array representation  -> The examples above will become

```json
    ["int32", [{":array":[1, 2]}, {":array":[3, 4]}]]
    ["int32", ["[1, 2]", "[3, 4]"]]
    ["int32", [2], [1, 2, 3, 4]]
```

option 3 : use JSON_object representation  -> The examples above will become

```json
    {"type": "int32", "data": [{":array":[1, 2]}, {":array":[3, 4]}]]}
    {"type": "int32", "data": ["[1, 2]", "[3, 4]"]]}
    {"type": "int32", "shape": [2], "data": [1, 2, 3, 4]]]
```


### Additional data

Multidimensional data is associated with complementary data :
- Name of axes (one axis per dimension)
- Axis values (optional)
- Additional variables associated with one or more axes (optional)
- Metadata (optional)
- name (optional)

The JSON representation must take this additional data into account.

Two solutions :
- solution 1 : structure multi-dimensional data + additional data
```json
 {'data'   : ['int32', [2, 3, 2, 2], 
              [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]], 
  'dims'   : ['x', 'y', 'z', 'option'],
  'coords': {'x': [['x1', 'x2']] , 
             'y': [['y1', 'y2', 'y3']],
             'z': ['string', ['z1', 'z2']],
             'option': [[True, False]],
             'xy': {'dims': ['x','y'], 'data': [[2,3], ['x1y1','x1y2','x1y3','x2y1','x2y2','x2y3']]}, 
             'opt_num': {'dims': ['option'], 'data': [[0, 1]]}},
   'attrs'  : {'meta': 'everything'}}
```
- solution 2 : unique structure for multi-dimensional data and additional data
```json
 {'type'   : 'int32',
  'shape'  : [2, 3, 2, 2],
  'data'   : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24], 
  'dims'   : ['x', 'y', 'z', 'option']
  'dimsco' : [['x1', 'x2'],['y1', 'y2', 'y3'], ['z1', 'z2'], [True, False]],
  'coords' : {'xy':{'dims': ['x', 'y'], 'data': [[2,3], ['x1y1', 'x1y2', 'x1y3', 'x2y1', 'x2y2', 'x2y3']]}, 
              'opt_num': {'dims': 'option', 'data': [0, 1]}},
  'attrs'  : {'meta': 'everything'}}
```

### NTV representation

Both structures can be represented with 
- NTVtype : `ndarray` or `xndarray`
- NTVname : name of the structure (optional)
- NTVvalue : JSON structure of `ndarray` or `xndarray`

Example:

```json
{ ':ndarray': ndarray_structure }
{ 'test:xndarray': xndarray_structure }
```

### Structure retained

The proposal is to retain two structures:
- `ndarray` to represent the data without additional data. This structure is the simplest and most compact. 

    `ndarray` is a JSON-array composed of :
    - values of data (JSON-array of flattened data)
    - shape of the data (JSON-array of axis length) - optional if the dimension is 1
    - type of data (string) - optional if the type is implicit

 ```json
              ["int32", [2, 2], [1, 2, 3, 4]]
              ["int32", [1, 2, 3, 4]]
              [[2, 2], [1, 2, 3, 4]]
              [[1, 2, 3, 4]]
 ```
 
- `xndarray` to represent data and additional data. This structure completes the `ndarray` structure.

    `xndarray` is a JSON-object composed of :

    - `data` (JSON-ndarray): values of data (see above)
    - `dims` (JSON-array): Name of axis (one axe per dimension) - optional
    - `coords` (JSON-object of `ndarray` or `xndarray`): Additional variables associated with one or more axis - optional
    - `attrs` (JSON-object): Metadata - optional
    - `name` (string): name - optional
        
 ```json
{'example1:ndarray':
   ['int32', [2, 3, 2], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]
}
{'example2:xndarray':
    {'data'  : ['int32', [2, 3, 2], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]], 
     'dims'   : ['x', 'y', 'option'],
     'coords' : {
         'x': [['x1', 'x2']] , 
         'y': ['string', ['y1', 'y2', 'y3']],
         'option': [[True, False]],
         'xy': {'dims': ['x','y'], 'data': [[2,3], ['x1y1','x1y2','x1y3','x2y1','x2y2','x2y3']]}, 
         'opt_num': {'dims': ['option'], 'data': [[0, 1]]}},
     'attrs'  : {'meta': 'everything'}}
}
```